In [1]:
import pandas as pd

In [2]:
df_iris = pd.read_csv(
    "https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/iris.csv",
    sep=',',
    thousands=None,
    decimal='.')
df_iris.head()

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
df_iris.describe()

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [4]:
df_iris.dtypes

Sepal_Length    float64
Sepal_Width     float64
Petal_Length    float64
Petal_Width     float64
Species          object
dtype: object

In [5]:
class Iris(object):
  def __init__(self, Sepal_Length, Sepal_Width, Petal_Length,Petal_Width,Species):
    self.Sepal_Length=Sepal_Length,
    self.Sepal_Width=Sepal_Width,
    self.Petal_Length=Petal_Length,
    self.Petal_Width=Petal_Width,
    self.Species=Species

#RavenDB
##Ejemplos

In [6]:
#Instalación del cliente - https://pypi.org/project/pyravendb/
!pip install pyravendb

     |████████████████████████████████| 122kB 5.3MB/s 
     |████████████████████████████████| 61kB 3.5MB/s 
     |████████████████████████████████| 61kB 4.3MB/s 
     |████████████████████████████████| 3.2MB 6.4MB/s 


Antes de continuar se crearán datos de ejemplo - https://ravendb.net/docs/article-page/4.2/java/studio/database/tasks/create-sample-data

##DocumentStore
Se debe crear una instancia de DocumentStorage, es el punto de entrada principal, es el responsable de establecer y administrar las conexiones entre el servidor o cluster y nuestra aplicación

In [7]:
from pyravendb.store import document_store


In [8]:
store = document_store.DocumentStore(urls=['http://live-test.ravendb.net'], database='DB-SAMPLE')
store.initialize()

DocumentStore es capaz de trabajar con multiples bases de datos, para un funcionamiento adecuado se recomienda una sola instancia por aplicación

##Session
Session se utiliza para manipular los datos, a diferencia de DocumentStore es ligero y se puede crear con más frecuencia, se recomienda crear sesiones por solicitud.

###Ejemplo1 - CREATE
RavenDB es una base de datos de documentos. Todos los objetos almacenados se denominan <b><i>documents</i></b>. Cada documentos contiene un <b>ID</b> unico que lo identifica, datos y metadatos adyacentes, ambos almacenados en formato JSON.

In [10]:
with store.open_session() as session:
  for i in range(len(df_iris)):
    session.store(Iris(
      df_iris.loc[i,'Sepal_Length'], 
      df_iris.loc[i,'Sepal_Width'],
      df_iris.loc[i,'Petal_Length'],
      df_iris.loc[i,'Petal_Width'],
      df_iris.loc[i,'Species'],
      
      ),key="iris/"+str(i))
  session.save_changes()

###Ejemplo2 - READ


In [12]:
with store.open_session() as session:
  iris = session.load("iris/100",object_type=Iris)
  print(iris.Species)

  irisList = ['iris/100','iris/99','iris/1','iris/0']
  irises = session.load(irisList)
  [print(x.Species) for x in irises]
  

virginica
virginica
versicolor
setosa
setosa


##Ejemplo3 - UPDATE
El metodo save_changes sincronizará (con el servidor) solo los documentos que se han cambiado dentro de la sesión. Todos los cambios se envían en una solicitud (ahorro de llamadas de red) y se procesan en una transacción.

In [ ]:
with store.open_session() as session:
  iris = session.load("iris/100", object_type=Iris)
  iris.Species = 'virginica'
  session.save_changes()

##Ejemplo4 - DELETE

In [ ]:
with store.open_session() as session:
  session.delete("iris/100")
  session.save_changes()

In [ ]:
with store.open_session()as session:
  session.store(Iris(
      df_iris.loc[100,'Sepal_Length'], 
      df_iris.loc[100,'Sepal_Width'],
      df_iris.loc[100,'Petal_Length'],
      df_iris.loc[100,'Petal_Width'],
      df_iris.loc[100,'Species'],
      
      ),key="iris/"+str(100))
  session.save_changes()

##Ejemplo5 - *Query*

Cuando no se especifica ningún índice en la consulta (como en la consulta a continuación), RavenDB utilizará su función de autoindexación inteligente que utilizará un índice ya existente o creará uno nuevo si no se encuentra ninguna coincidencia. La otra opción es escribir el índice usted mismo e implementarlo en el servidor. Esos índices se denominan índices estáticos.

In [13]:
with store.open_session() as session:
  housing_list = list(
      session
        .query(object_type=Iris)
        .where(Species='setosa')
        .select('Sepal_Length')

  )
  [print(x.Sepal_Length) for x in housing_list]

([5.1],)
([4.9],)
([4.7],)
([4.6],)
([5.0],)
([5.4],)
([4.6],)
([5.0],)
([4.4],)
([4.9],)
([5.4],)
([4.8],)
([4.8],)
([4.3],)
([5.8],)
([5.7],)
([5.4],)
([5.1],)
([5.7],)
([5.1],)
([5.4],)
([5.1],)
([4.6],)
([5.1],)
([4.8],)
([5.0],)
([5.0],)
([5.2],)
([5.2],)
([4.7],)
([4.8],)
([5.4],)
([5.2],)
([5.5],)
([4.9],)
([5.0],)
([5.5],)
([4.9],)
([4.4],)
([5.1],)
([5.0],)
([4.5],)
([4.4],)
([5.0],)
([5.1],)
([4.8],)
([5.1],)
([4.6],)
([5.3],)
([5.0],)


##Ejemplo6 - MapReduce

Conteo y Ganancias por Proveedor y categoría, siempre y cuando el precio por producto sea mayor a 10 y el producto no esté descontinuado.

```
# MapReduce
// Map Function:
map('Products', (product) => { 
    if (product.PricePerUnit > 10 && product.Discontinued !== false) {
        return { 
            Supplier: product.Supplier, 
            Category: product.Category,
            Count: 1,
            Revenue: product.UnitsOnOrder * product.PricePerUnit
        };
    }
})

// Reduce Function:
groupBy(x => ({ Supplier: x.Supplier, Category: x.Category }))
.aggregate(g => { 
    return {
        Supplier: g.key.Supplier,
        Category: g.key.Category,
        Count: g.values.reduce((count, val) => val.Count + count, 0),
        Revenue: g.values.reduce((amount, val) => val.Revenue + amount, 0)
    };
})
```



##Bibliografia
https://hub.docker.com/r/ravendb/ravendb/<br>
https://ravendb.net/docs/article-page/4.2/java/start/getting-started#client<br>
https://ravendb.net/docs/article-page/4.2/java/studio/database/indexes/create-map-reduce-index<br>
https://github.com/ravendb/ravendb-python-client<br>


#Ejercicios

##1 -  El programa MapReduce debe imprimir cada especie de flor (setosa, versicolor, virginica) y la cantidad de documentos con dicha especie


In [2]:
df_ejercicio1 = pd.read_csv('ejercicio1.csv')
df_ejercicio1

,Species,Count
0,setosa,50
1,versicolor,50
2,virginica,50


##2 - El programa MapReduce debe imprimir cada una de las categorías existentes de la colección <b>Products</b>, la cantidad de productos por categoría y las ganancias asociadas a cada categoría (Precio de cada unidad x unidades en la orden)

In [3]:
df_ejercicio2=pd.read_csv('ejercicio2.csv')
df_ejercicio2

,Category,Count,Revenue
0,categories/1-A,10,11421.50
1,categories/2-A,11,11893.55
2,categories/7-A,4,3509.35
3,categories/6-A,5,5573.00
4,categories/8-A,9,11998.60
5,categories/4-A,9,10991.20
6,categories/3-A,10,9790.20
7,categories/5-A,5,4779.50


##3 - El programa MapReduce debe imprimir Pais y Ciudad de la colección <b>Orders</b>, la cantidad de productos por orden realizada, las ganancias obtenidas en esa orden y la cantidad de ordenes realizadas.

In [4]:
df_ejercicio3=pd.read_csv('ejercicio3.csv')
df_ejercicio3

,Country,City,Quantity,Revenue,Count
0,France,Reims,98,1480.00,5
1,Germany,Münster,253,4954.00,6
2,Brazil,Rio de Janeiro,1893,53999.18,34
3,France,Lyon,434,9937.10,10
4,Belgium,Charleroi,1072,24704.40,12
...,...,...,...,...,...
65,Finland,Helsinki,148,3161.35,7
66,USA,Butte,59,1947.24,3
67,Germany,Berlin,174,4596.20,6
68,France,Paris,48,2423.35,4
